# Ideas for Metrics

## Theorize Components of Tackle
* Important parts of a tackle:
    - Reaching the same place - can’t make a tackle from looking at a ball carrier, must make contact
        * speed, direction, angle, persuit
        * how far away is the tackler from the ball carrier? (distance)
        * if both ball carrier and tackler maintain their direction and speed where will their paths cross? (distance to contact point)
        * speed and direction changes
            - juking, cutting, throwing on the breaks as techniques for avoiding tackles
            - defenders reaction time, ability to adjust to changes
    - Force at contact
        * when the tackler and the defender meet the force applied through the two players will be impacted by each one's speed, direction, size, etc
            - height and weight differential
            - angle and speed differential
                * same speed, same angle/direction will impart much less force than same speed opposite angle/direction
                * defenders will not have much luck making good tackles if they are stationary at the point of contact
    - Form (rugby, all of these are from a mostly head on collision type attack angle)
        * get low 
            - contact through the hips of the ball carrier
            - cheek to cheek (face to butt)
            - pick a side (don't put your head in the middle of the ball carrier's body)
            - chop your feet just before contact (the decrease in speed of your lower body and momentum of your upper body helps drop the height of your head and shoulders to improve contact position without losing all your momentum)
        * wrap arms around the ball carrier
            - front arm around mid section prevents the ball carrier from off loading to a support player
            - back arm wraps around back of leg, picking up that leg and driving prevents the ball carrier from continuing to drive their legs and make forward progress
        * drive through the ball carrier
            - make contact with the ball carrier as if you want to drive them backward into someone 1 yard behind them
    - Access
        * difficult to tackle the ball carrier if there is a blocker between the defender and the ball carrier

## Single Frame Metrics
- make contact metrics
    - tackler distance to ball carrier
    - tackler distance to contact point
        - using the direction of each player, find the point where they will meet, calculate the distance between the tackler and that point
    - tackler time to contact
        - see above calculation
        - use tackler speed to determine time to cover the distance
    - ball carrier time to contact
        - see above, but ball carrier
    - difference in time to contact
        - find the difference between tackler time to contact and ball carrier time to contact
    - will make contact yes/no
        - set a limit for the difference in time to contact point
        - example: assume reaching the same point 2 seconds appart will result in the tackler not making contact with the ball carrier
            - need to determine how to set the max time difference
- force metrics
    - tackler force
        - total (mass * acceleration)
        - north-south (trigonometry with force and direction)
        - east_west 
        - in direction of contact point
    - ball carrier force 
        - total, north-south, east-west, contact
    - force difference
        - total, north-south, east-west, contact
        - adjust total based on angle of contact
            - reduce the force metric based on the angle of tackler and ball carrier contact
                - possible calculation: determine the force of the tackler in the direction the ball carrier is moving, compare that to the full force of the ball carrier
- direction and angle
    - difference in direction of movement and orientation
        - normalize using trigonometry?

## Multi-Frame Metrics
 - other metrics that feel outside of the ability to evaluate from a single frame and may be better if aggregating over mulitple frames (will define later)
    - make contact metrics
        - ball carrier shiftiness
        - tackler adjustments
    - force metrics
        - conservation of momentum

## Calculate Force Metrics
Starting with force metrics as a gentler entry into the trigonometry

Force = mass * acceleration

Momentum = mass * velocity (speed + direction)

**Step 1: momentum/force in direction of player movement**
- join player weight to tracking data
- multiply weight by speed and acceleration

**Notes:**
- adjusting units to kgs and meters may make the values more accessible to individuals familiar with force/momentum calculations but should have no impact on the distribution analysis
    - player weights is most likely in pounds
    - speed in yards per second
    - acceleration in yards per second squared

In [1]:
import pandas as pd
import math
import numpy as np

In [ ]:
# OPTIONAL: Load the "autoreload" extension so that code can change
%load_ext autoreload

# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

from src.data.tackler_info import (
    simplify_tackles_df, join_ball_carrier_tracking, player_dist_to_ball_carrier, 
    tackler_distance, dist_group
)
from src.visualization.visualize import show_frame, animate_play

In [2]:
games_df = pd.read_csv('../data/raw/games.csv')
plays_df = pd.read_csv('../data/raw/plays.csv')
players_df = pd.read_csv('../data/raw/players.csv')
tackles_df = pd.read_csv('../data/raw/tackles.csv')
tracking_week_1_df = pd.read_csv('../data/raw/tracking_week_1.csv')

In [ ]:
players_df.head()

In [ ]:
weight_df = tracking_week_1_df.merge(players_df[['nflId', 'weight']], on='nflId')

In [ ]:
weight_df['momentum'] = weight_df['s'] * weight_df['weight']
weight_df['force'] = weight_df['a'] * weight_df['weight']

In [ ]:
weight_df.head()

**Step 2: momentum/force in y direction**
- force in y direction = force * cosine(direction)
- force in x direction = force * sine(direction)

**Notes:**
- direciton is the angle of player motion
    - it is relative to the y axis so 0 is moving from the home sideline to the visitor sideline
    - direction is in degrees and needs to be converted to radians before calculating the cosine or sine

In [ ]:
weight_df['dir_cos'] = np.cos(np.radians(weight_df['dir']))
weight_df['dir_sin'] = np.sin(np.radians(weight_df['dir']))

In [ ]:
weight_df['force_y'] = weight_df['force'] * weight_df['dir_cos']
weight_df['force_x'] = weight_df['force'] * weight_df['dir_sin']
weight_df['momentum_y'] = weight_df['momentum'] * weight_df['dir_cos']
weight_df['momentum_x'] = weight_df['momentum'] * weight_df['dir_sin']

In [ ]:
# verify positive and negative force/momentum align with expected directions 
# - all calculated dataframes shoud be empty

# first quadrant, expect positive force in x and y directions
assert weight_df[(weight_df['dir'] < 90) & ((weight_df['force_x'] < 0) | (weight_df['force_y'] < 0))].empty

# second quadrant, expect positive force in x direction and negative in y direction
assert weight_df[((90 < weight_df['dir']) & (weight_df['dir'] < 180)) & ((weight_df['force_x'] < 0) | (weight_df['force_y'] > 0))].empty

# third quadrant, expect negative force in x and y directions
assert weight_df[((180 < weight_df['dir']) & (weight_df['dir'] < 270)) & ((weight_df['force_x'] > 0) | (weight_df['force_y'] > 0))].empty

# fourth quadrant, expect negative force in x direction and positive in y direction
assert weight_df[(weight_df['dir'] > 270) & ((weight_df['force_x'] > 0) | (weight_df['force_y'] < 0))].empty

**Step 3: Calculate differences between ball carrier and defender**
- ball carrier data merged into every frame
- differences calculated on total, x, and y directional force and momentum

## Calculate Point of Contact
- Use direction of player movement (dir)

In [ ]:
# equation of a line - y = mx + b
# Step 1 - find slope
    # change angle to counter clockwise from x-axis instead of clockwise from y-axis
    # theta = 90 - dir
    # m = tan(theta)
# Step 2 - solve for intercept
    # b = y - (m * x)
# Step 3 - find x intersection
    # m_bc * x_cross + b_bc = m_t * x_cross + b_t
    # (m_bc * x_cross) - (m_t * x_cross) = b_t - b_bc
    # x_cross * (m_bc - m_t) = b_t - b_bc
    # x_cross = (b_t - b_bc) / (m_bc - m_t)
# Step 4 - find y intersection
    # y_cross = m_bc * x_cross + b_bc
# Step 5 - check
    # y_cross = m_t * x_cross + b_t
    
# Abbreviated Version - find x
    #x_cross = ((y_t - (tan(dir_t) * x_t)) - (y_bc - (tan(dir_bc) * x_bc)) / (tan(dir_bc) - tan(dir_t))
    #y_cross = tan(dir_ball_carrier) * x_cross + (y_ball_carrier - (tan(dir_ball_carrier) * x_ball_carrier)

### Check initial dataframe

#1 Field Direction Normalization

In [ ]:
animate_play(tracking_week_1_df, plays_df, players_df, gameId=2022090800, playId=56)

In [ ]:
physics_df = pd.read_csv('../data/interim/physics_tackle_v1_check.csv').head(len(tracking_week_1_df))

In [ ]:
animate_play(physics_df, plays_df, players_df, gameId=2022090800, playId=56)

**Run all Dataframe calculations**

In [3]:
# OPTIONAL: Load the "autoreload" extension so that code can change
%load_ext autoreload

# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

from src.data.tackler_info import (
    simplify_tackles_df, join_ball_carrier_tracking,
    player_dist_to_ball_carrier, tackler_distance_frame
)
from src.data.physics import (
    calculate_angles, physics_calculations, find_contact_point,
    metric_diffs
)
from src.data.make_dataset import (
    standardize_field
)

In [28]:
tracking_df = tracking_week_1_df

In [29]:
standard_tracking_df = standardize_field(tracking_df)

In [30]:
# merge weight into tracking dataframe
weight_df = standard_tracking_df.merge(players_df[['nflId', 'weight']], on='nflId')

In [31]:
# add force, momentum, and angles
angle_tracking_df = calculate_angles(weight_df)
force_tracking_df = physics_calculations(angle_tracking_df, "force")
physics_tracking_df = physics_calculations(force_tracking_df, "momentum")

In [32]:
# add ball carrier details to every row
tackle_simple_df = simplify_tackles_df(tackles_df)
ball_carrier_join_df = join_ball_carrier_tracking(plays_df, physics_tracking_df)

In [33]:
# calculate distances and select single frame
ball_carrier_dist_df = player_dist_to_ball_carrier(ball_carrier_join_df)

In [34]:
tackler_dist_df = tackler_distance_frame(tackle_simple_df, ball_carrier_dist_df, 2)

In [35]:
# calculate metrics differences
force_diff_df = metric_diffs(tackler_dist_df, 'force')
momentum_force_diff_df = metric_diffs(force_diff_df, 'momentum')

In [37]:
# calculate contact point
contact_point_df = find_contact_point(momentum_force_diff_df)

**Sanity check**

In [16]:
contact_point_df[['club', 'club_ball_carrier', 'nflId', 'nflId_ball_carrier', 'dir', 'slope', 'x', 'y', 'dir_ball_carrier', 'slope_ball_carrier', 'x_ball_carrier', 'y_ball_carrier', 'x_contact', 'y_contact', 'contact_y_check']]

,club,club_ball_carrier,nflId,nflId_ball_carrier,dir,slope,x,y,dir_ball_carrier,slope_ball_carrier,x_ball_carrier,y_ball_carrier,x_contact,y_contact,contact_y_check
0,LA,BUF,38577.0,42489.0,207.56,1.916080,41.47,21.183333,267.79,0.038591,40.73,19.493333,40.585072,19.487740,19.487740
1,BUF,BUF,42489.0,42489.0,369.38,6.053624,39.40,15.043333,369.38,6.053624,39.40,15.043333,39.400000,15.043333,15.043333
2,LA,BUF,43294.0,42489.0,357.68,-24.682958,41.80,17.553333,387.99,1.881519,40.69,18.933333,41.669431,20.776152,20.776152
3,LA,BUF,53532.0,42489.0,266.55,0.060287,42.52,19.973333,385.93,2.056677,40.77,19.103333,41.152940,19.890918,19.890918
4,LA,BUF,43294.0,46076.0,384.06,2.239722,48.00,8.783333,509.26,-1.681517,47.16,10.343333,48.037622,8.867597,8.867597
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9836,SEA,DEN,44873.0,53464.0,337.74,-2.443109,63.37,44.623333,492.36,-0.911846,62.27,45.363333,63.541774,44.203671,44.203671
9837,SEA,DEN,52435.0,53464.0,313.45,-0.947307,62.58,45.903333,483.65,-0.665657,61.20,46.183333,64.847374,43.755433,43.755433
9838,SEA,DEN,52456.0,53464.0,394.81,1.438276,64.89,42.173333,512.84,-1.949135,63.71,43.373333,64.565274,41.706288,41.706288
9839,DEN,DEN,53464.0,53464.0,358.63,-41.813766,49.29,39.433333,358.63,-41.813766,49.29,39.433333,49.290000,39.433333,39.433333


In [ ]:
contact_point_df[['min_dist', 'force_diff', 'force_x_diff',
       'force_y_diff', 'momentum_diff', 'momentum_x_diff', 'momentum_y_diff']]

## Contact Metrics
- make contact metrics
    - ~~tackler distance to ball carrier
    - ~~tackler distance to contact point
        - using the direction of each player, find the point where they will meet, calculate the distance between the tackler and that point
    - ~~tackler time to contact
        - see above calculation
        - use tackler speed to determine time to cover the distance
    - ~~ball carrier time to contact
        - see above, but ball carrier
    - ~~difference in time to contact
        - find the difference between tackler time to contact and ball carrier time to contact
    - will make contact yes/no
        - set a limit for the difference in time to contact point
        - example: assume reaching the same point 2 seconds appart will result in the tackler not making contact with the ball carrier
            - need to determine how to set the max time difference


### Distance to contact point

In [17]:
from scipy.spatial.distance import euclidean

In [18]:
def dist_calc(df, first='', second='_ball_carrier', name='tackler_to_ball_carrier_dist'):
    # calculate distance from each player to ball carrier
    df['distance'] = df.apply(
                    lambda row: euclidean((row[f'x{first}'], row[f'y{first}']), 
                                          (row[f'x{second}'], row[f'y{second}']))
                    , axis=1)
    df.rename({'distance': name}, axis=1, inplace=True)
    return df

In [19]:
dist_calc(contact_point_df)

,gameId,playId,event,nflId,frameId,displayName,club,x,y,s,...,force_diff,force_x_diff,force_y_diff,momentum_diff,momentum_x_diff,momentum_y_diff,x_contact,y_contact,contact_y_check,tackler_to_ball_carrier_dist
0,2022090800,56,None,38577.0,17,Bobby Wagner,LA,41.47,21.183333,2.03,...,-429.96,20.606051,662.983618,-388.12,124.231534,431.537852,40.585072,19.487740,19.487740,1.844912
1,2022090800,56,None,42489.0,1,Stefon Diggs,BUF,39.40,15.043333,6.23,...,0.00,0.000000,0.000000,0.00,0.000000,0.000000,39.400000,15.043333,15.043333,0.000000
2,2022090800,56,tackle,43294.0,10,Jalen Ramsey,LA,41.80,17.553333,4.37,...,178.56,506.892903,61.277872,-490.67,233.106020,-538.852532,41.669431,20.776152,20.776152,1.771017
3,2022090800,56,None,53532.0,11,Ernest Jones,LA,42.52,19.973333,1.54,...,859.96,501.496303,857.247783,-39.05,491.364584,304.738381,41.152940,19.890918,19.890918,1.954329
4,2022090800,80,None,43294.0,27,Jalen Ramsey,LA,48.00,8.783333,0.59,...,741.13,432.832650,-1562.444662,1000.66,524.175822,-1077.598174,48.037622,8.867597,8.867597,1.771779
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9836,2022091200,3826,None,44873.0,39,Josh Jones,SEA,63.37,44.623333,4.47,...,-822.80,1403.489575,-2323.638569,-151.80,987.012689,-1470.432535,63.541774,44.203671,44.203671,1.325745
9837,2022091200,3826,tackle,52435.0,36,Jordyn Brooks,SEA,62.58,45.903333,2.09,...,-1291.15,1609.329957,-1423.432557,638.55,1329.538556,-989.716094,64.847374,43.755433,43.755433,1.408119
9838,2022091200,3826,None,52456.0,47,Darrell Taylor,SEA,64.89,42.173333,2.94,...,-614.13,-367.439944,-756.400872,-116.82,-138.403754,-1168.577120,64.565274,41.706288,41.706288,1.682974
9839,2022091200,3826,None,53464.0,1,Javonte Williams,DEN,49.29,39.433333,4.01,...,0.00,0.000000,0.000000,0.00,0.000000,0.000000,49.290000,39.433333,39.433333,0.000000


In [20]:
dist_calc(contact_point_df, first='', second='_contact', name='tackler_to_contact_dist')

,gameId,playId,event,nflId,frameId,displayName,club,x,y,s,...,force_x_diff,force_y_diff,momentum_diff,momentum_x_diff,momentum_y_diff,x_contact,y_contact,contact_y_check,tackler_to_ball_carrier_dist,tackler_to_contact_dist
0,2022090800,56,None,38577.0,17,Bobby Wagner,LA,41.47,21.183333,2.03,...,20.606051,662.983618,-388.12,124.231534,431.537852,40.585072,19.487740,19.487740,1.844912,1.912625e+00
1,2022090800,56,None,42489.0,1,Stefon Diggs,BUF,39.40,15.043333,6.23,...,0.000000,0.000000,0.00,0.000000,0.000000,39.400000,15.043333,15.043333,0.000000,1.421085e-14
2,2022090800,56,tackle,43294.0,10,Jalen Ramsey,LA,41.80,17.553333,4.37,...,506.892903,61.277872,-490.67,233.106020,-538.852532,41.669431,20.776152,20.776152,1.771017,3.225462e+00
3,2022090800,56,None,53532.0,11,Ernest Jones,LA,42.52,19.973333,1.54,...,501.496303,857.247783,-39.05,491.364584,304.738381,41.152940,19.890918,19.890918,1.954329,1.369542e+00
4,2022090800,80,None,43294.0,27,Jalen Ramsey,LA,48.00,8.783333,0.59,...,432.832650,-1562.444662,1000.66,524.175822,-1077.598174,48.037622,8.867597,8.867597,1.771779,9.228091e-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9836,2022091200,3826,None,44873.0,39,Josh Jones,SEA,63.37,44.623333,4.47,...,1403.489575,-2323.638569,-151.80,987.012689,-1470.432535,63.541774,44.203671,44.203671,1.325745,4.534559e-01
9837,2022091200,3826,tackle,52435.0,36,Jordyn Brooks,SEA,62.58,45.903333,2.09,...,1609.329957,-1423.432557,638.55,1329.538556,-989.716094,64.847374,43.755433,43.755433,1.408119,3.123213e+00
9838,2022091200,3826,None,52456.0,47,Darrell Taylor,SEA,64.89,42.173333,2.94,...,-367.439944,-756.400872,-116.82,-138.403754,-1168.577120,64.565274,41.706288,41.706288,1.682974,5.688393e-01
9839,2022091200,3826,None,53464.0,1,Javonte Williams,DEN,49.29,39.433333,4.01,...,0.000000,0.000000,0.00,0.000000,0.000000,49.290000,39.433333,39.433333,0.000000,5.684342e-14


In [21]:
dist_calc(contact_point_df, first='_ball_carrier', second='_contact', name='ball_carrier_to_contact_dist')

,gameId,playId,event,nflId,frameId,displayName,club,x,y,s,...,force_y_diff,momentum_diff,momentum_x_diff,momentum_y_diff,x_contact,y_contact,contact_y_check,tackler_to_ball_carrier_dist,tackler_to_contact_dist,ball_carrier_to_contact_dist
0,2022090800,56,None,38577.0,17,Bobby Wagner,LA,41.47,21.183333,2.03,...,662.983618,-388.12,124.231534,431.537852,40.585072,19.487740,19.487740,1.844912,1.912625e+00,1.450357e-01
1,2022090800,56,None,42489.0,1,Stefon Diggs,BUF,39.40,15.043333,6.23,...,0.000000,0.00,0.000000,0.000000,39.400000,15.043333,15.043333,0.000000,1.421085e-14,1.421085e-14
2,2022090800,56,tackle,43294.0,10,Jalen Ramsey,LA,41.80,17.553333,4.37,...,61.277872,-490.67,233.106020,-538.852532,41.669431,20.776152,20.776152,1.771017,3.225462e+00,2.086927e+00
3,2022090800,56,None,53532.0,11,Ernest Jones,LA,42.52,19.973333,1.54,...,857.247783,-39.05,491.364584,304.738381,41.152940,19.890918,19.890918,1.954329,1.369542e+00,8.757468e-01
4,2022090800,80,None,43294.0,27,Jalen Ramsey,LA,48.00,8.783333,0.59,...,-1562.444662,1000.66,524.175822,-1077.598174,48.037622,8.867597,8.867597,1.771779,9.228091e-02,1.716980e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9836,2022091200,3826,None,44873.0,39,Josh Jones,SEA,63.37,44.623333,4.47,...,-2323.638569,-151.80,987.012689,-1470.432535,63.541774,44.203671,44.203671,1.325745,4.534559e-01,1.721111e+00
9837,2022091200,3826,tackle,52435.0,36,Jordyn Brooks,SEA,62.58,45.903333,2.09,...,-1423.432557,638.55,1329.538556,-989.716094,64.847374,43.755433,43.755433,1.408119,3.123213e+00,4.381556e+00
9838,2022091200,3826,None,52456.0,47,Darrell Taylor,SEA,64.89,42.173333,2.94,...,-756.400872,-116.82,-138.403754,-1168.577120,64.565274,41.706288,41.706288,1.682974,5.688393e-01,1.873642e+00
9839,2022091200,3826,None,53464.0,1,Javonte Williams,DEN,49.29,39.433333,4.01,...,0.000000,0.00,0.000000,0.000000,49.290000,39.433333,39.433333,0.000000,5.684342e-14,5.684342e-14


### Time to Contact Point

In [24]:
def time_to_contact(df):
    df['tackler_time_to_contact'] = df['tackler_to_contact_dist'] / df['s']
    df['ball_carrier_time_to_contact'] = df['ball_carrier_to_contact_dist'] / df['s_ball_carrier']
    df['diff_time_to_contact'] = df['tackler_time_to_contact'] - df['ball_carrier_time_to_contact']
    return df

In [25]:
time_to_contact(contact_point_df)
contact_point_df

,gameId,playId,event,nflId,frameId,displayName,club,x,y,s,...,momentum_y_diff,x_contact,y_contact,contact_y_check,tackler_to_ball_carrier_dist,tackler_to_contact_dist,ball_carrier_to_contact_dist,tackler_time_to_contact,ball_carrier_time_to_contact,diff_time_to_contact
0,2022090800,56,None,38577.0,17,Bobby Wagner,LA,41.47,21.183333,2.03,...,431.537852,40.585072,19.487740,19.487740,1.844912,1.912625e+00,1.450357e-01,9.421796e-01,2.685846e-01,0.673595
1,2022090800,56,None,42489.0,1,Stefon Diggs,BUF,39.40,15.043333,6.23,...,0.000000,39.400000,15.043333,15.043333,0.000000,1.421085e-14,1.421085e-14,2.281036e-15,2.281036e-15,0.000000
2,2022090800,56,tackle,43294.0,10,Jalen Ramsey,LA,41.80,17.553333,4.37,...,-538.852532,41.669431,20.776152,20.776152,1.771017,3.225462e+00,2.086927e+00,7.380921e-01,9.529349e-01,-0.214843
3,2022090800,56,None,53532.0,11,Ernest Jones,LA,42.52,19.973333,1.54,...,304.738381,41.152940,19.890918,19.890918,1.954329,1.369542e+00,8.757468e-01,8.893129e-01,5.307556e-01,0.358557
4,2022090800,80,None,43294.0,27,Jalen Ramsey,LA,48.00,8.783333,0.59,...,-1077.598174,48.037622,8.867597,8.867597,1.771779,9.228091e-02,1.716980e+00,1.564083e-01,3.622320e-01,-0.205824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9836,2022091200,3826,None,44873.0,39,Josh Jones,SEA,63.37,44.623333,4.47,...,-1470.432535,63.541774,44.203671,44.203671,1.325745,4.534559e-01,1.721111e+00,1.014443e-01,4.553204e-01,-0.353876
9837,2022091200,3826,tackle,52435.0,36,Jordyn Brooks,SEA,62.58,45.903333,2.09,...,-989.716094,64.847374,43.755433,43.755433,1.408119,3.123213e+00,4.381556e+00,1.494360e+00,8.377736e-01,0.656587
9838,2022091200,3826,None,52456.0,47,Darrell Taylor,SEA,64.89,42.173333,2.94,...,-1168.577120,64.565274,41.706288,41.706288,1.682974,5.688393e-01,1.873642e+00,1.934827e-01,6.574182e-01,-0.463935
9839,2022091200,3826,None,53464.0,1,Javonte Williams,DEN,49.29,39.433333,4.01,...,0.000000,49.290000,39.433333,39.433333,0.000000,5.684342e-14,5.684342e-14,1.417542e-14,1.417542e-14,0.000000


In [27]:
momentum_force_diff_df

,gameId,playId,event,nflId,frameId,displayName,club,x,y,s,...,momentum_y_diff,x_contact,y_contact,contact_y_check,tackler_to_ball_carrier_dist,tackler_to_contact_dist,ball_carrier_to_contact_dist,tackler_time_to_contact,ball_carrier_time_to_contact,diff_time_to_contact
0,2022090800,56,None,38577.0,17,Bobby Wagner,LA,41.47,21.183333,2.03,...,431.537852,40.585072,19.487740,19.487740,1.844912,1.912625e+00,1.450357e-01,9.421796e-01,2.685846e-01,0.673595
1,2022090800,56,None,42489.0,1,Stefon Diggs,BUF,39.40,15.043333,6.23,...,0.000000,39.400000,15.043333,15.043333,0.000000,1.421085e-14,1.421085e-14,2.281036e-15,2.281036e-15,0.000000
2,2022090800,56,tackle,43294.0,10,Jalen Ramsey,LA,41.80,17.553333,4.37,...,-538.852532,41.669431,20.776152,20.776152,1.771017,3.225462e+00,2.086927e+00,7.380921e-01,9.529349e-01,-0.214843
3,2022090800,56,None,53532.0,11,Ernest Jones,LA,42.52,19.973333,1.54,...,304.738381,41.152940,19.890918,19.890918,1.954329,1.369542e+00,8.757468e-01,8.893129e-01,5.307556e-01,0.358557
4,2022090800,80,None,43294.0,27,Jalen Ramsey,LA,48.00,8.783333,0.59,...,-1077.598174,48.037622,8.867597,8.867597,1.771779,9.228091e-02,1.716980e+00,1.564083e-01,3.622320e-01,-0.205824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9836,2022091200,3826,None,44873.0,39,Josh Jones,SEA,63.37,44.623333,4.47,...,-1470.432535,63.541774,44.203671,44.203671,1.325745,4.534559e-01,1.721111e+00,1.014443e-01,4.553204e-01,-0.353876
9837,2022091200,3826,tackle,52435.0,36,Jordyn Brooks,SEA,62.58,45.903333,2.09,...,-989.716094,64.847374,43.755433,43.755433,1.408119,3.123213e+00,4.381556e+00,1.494360e+00,8.377736e-01,0.656587
9838,2022091200,3826,None,52456.0,47,Darrell Taylor,SEA,64.89,42.173333,2.94,...,-1168.577120,64.565274,41.706288,41.706288,1.682974,5.688393e-01,1.873642e+00,1.934827e-01,6.574182e-01,-0.463935
9839,2022091200,3826,None,53464.0,1,Javonte Williams,DEN,49.29,39.433333,4.01,...,0.000000,49.290000,39.433333,39.433333,0.000000,5.684342e-14,5.684342e-14,1.417542e-14,1.417542e-14,0.000000
